# Import Necessary Libraries

In [1]:
import os
import gc
import copy

# Load all the necessary libraries
import numpy as np  # numerical computation with arrays
import pandas as pd # library to manipulate datasets using dataframes
import scipy as sp  # statistical library
from scipy import stats

# Load sklearn libraries for machine learning
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold ,RepeatedKFold, StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error


# Load plotting libraries
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
plt.style.use('ggplot')
import seaborn as sns

import shap

from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
!ls ..

# Load the Data

In [3]:
train_df = pd.read_csv("../input/widsdatathon2022/train.csv")
test_df = pd.read_csv("../input/widsdatathon2022/test.csv")

In [4]:
# !git branch -a
train_df.shape, test_df.shape

## Exploring Data (EDA) + Data Visualization

In [5]:
train_df.head()

In [6]:
train_df.info()

In [7]:
train_df.describe()

In [8]:
train_df.columns

In [9]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [10]:
# Looking at the distributions of all numerical variables
plt.figure(figsize=(20,20))

col = ['floor_area', 'year_built', 'energy_star_rating', 'ELEVATION', 'cooling_degree_days','heating_degree_days',
       'precipitation_inches', 'snowfall_inches', 'snowdepth_inches', 'avg_temp', 'days_below_30F', 'days_below_20F',
       'days_below_10F', 'days_below_0F', 'days_above_80F', 'days_above_90F', 'days_above_100F', 'days_above_110F',
       'direction_max_wind_speed', 'direction_peak_wind_speed', 'max_wind_speed', 'days_with_fog']

a = 1
for i in range(len(col)):
    plt.subplot(8, 3, a); 
    sns.distplot(train_df[col[i]])
    sns.distplot(test_df[col[i]])
    plt.xlabel(col[i])
    a += 1

plt.tight_layout()

# Differences - Elevation, cooling_degree_days, heating_degree_days, snowfall_inches, avg_temp, days_below_20F,
# days_above_80F, direction_max_wind_speed, direction_peak_wind_speed, max_wind_speed, days_with_fog

In [11]:
# Looking at the distributions of all temperature variables
plt.figure(figsize=(20,20))

months = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", \
          "november", "december"]
temp_cal = ["min", "avg", "max"]
a = 1
for i in range(len(months)):
    for j in temp_cal:
        plt.subplot(12, 3, a); 
        sns.distplot(train_df[months[i]+"_"+ j +"_temp"])
        sns.distplot(test_df[months[i]+"_"+ j +"_temp"])        
        plt.xlabel(months[i]+"_"+ j +"_temp")
        a += 1

plt.tight_layout()

# Differences - 

In [12]:
# Looking at the distributions of all discrete variables in train dataset
plt.figure(figsize=(20,15))
discrete = ['Year_Factor', 'State_Factor', 'building_class', 'facility_type', 'energy_star_rating', 'year_built']
a = 1
for i in range(len(discrete)):
    plt.subplot(3, 3, a); 
    sns.countplot(train_df[discrete[i]])
    plt.xlabel(discrete[i])
    a += 1
plt.tight_layout()

In [13]:
# Looking at the distributions of all discrete variables in test dataset
plt.figure(figsize=(20,15))
discrete = ['Year_Factor', 'State_Factor', 'building_class', 'facility_type', 'energy_star_rating', 'year_built']
a = 1
for i in range(len(discrete)):
    plt.subplot(3, 3, a); 
    sns.countplot(test_df[discrete[i]])
    plt.xlabel(discrete[i])
    a += 1
plt.tight_layout()

In [14]:
# Distribution of target variable
sns.distplot(train_df["site_eui"])
plt.xlabel("site_eui")

In [15]:
# Features having missing values in train data
train_df_na = pd.DataFrame(train_df.isna().sum())
train_df_na["total"] = train_df.shape[0]
train_df_na["perc_missing"] = round((train_df_na[0]/train_df_na["total"])*100,2)
train_df_na[train_df_na.iloc[:,0]>0]

In [16]:
# Features having missing values in test data
test_df_na = pd.DataFrame(test_df.isna().sum())
test_df_na["total"] = test_df.shape[0]
test_df_na["perc_missing"] = round((test_df_na[0]/test_df_na["total"])*100,2)
test_df_na[test_df_na.iloc[:,0]>0]

### Dealing with Missing Values

In [17]:
# Removing the 4 features above, since > 80% missing in test data
# removing days_above_110F since has only 0's as values
var_remove = ["direction_max_wind_speed", "direction_peak_wind_speed", "max_wind_speed", "days_with_fog", "days_above_110F"]
train_df = train_df.drop(columns=var_remove, axis=1)
test_df = test_df.drop(columns=var_remove, axis=1)
train_df.shape, test_df.shape

In [18]:
train_df.energy_star_rating.value_counts(), test_df.energy_star_rating.value_counts(), \
test_df.energy_star_rating.min()

In [20]:
# year_built: replace with 2000, which is the mode year in test dataset.
train_df['year_built'] =train_df['year_built'].replace(np.nan, 2000)
#replacing energy_star_rating missing values with median
train_df['energy_star_rating']=train_df['energy_star_rating'].replace(np.nan,train_df['energy_star_rating'].median())

# Replicating the above steps for test dataset
test_df['year_built'] =test_df['year_built'].replace(np.nan, 2000)
test_df['energy_star_rating']=test_df['energy_star_rating'].replace(np.nan,train_df['energy_star_rating'].median())

train_df.shape, test_df.shape

In [21]:
# Looking at the distribution of facility type variable in train dataset
plt.figure(figsize=(16,24))

sns.countplot(y="facility_type",data=train_df)
plt.title('Train')

In [22]:
# Looking at the distribution of facility type variable in test dataset
plt.figure(figsize=(16,24))

sns.countplot(y="facility_type",data=test_df)
plt.title('Test')

### Feature Preprocessing

In [23]:
num_features = train_df.select_dtypes(include=['float64']).columns.values

# Compute correlations between all numerical features

train_corr = train_df[num_features].corr()

In [24]:
# Sort features by their (absolute) correlation with the target variable.
# Here we see that "energy_star_rating" is highly correlated with site_eui (our target), so it might be a good predictor.

train_corr.sort_values(by='site_eui', key=abs, ascending=False)['site_eui']

In [25]:
# Plot correlations as a heatmap
# This could indicate that we can drop some of these features.

fig, ax = plt.subplots(figsize=(27, 30))
sns.heatmap(train_corr, vmin=-1, vmax=1, cmap = 'RdBu_r', xticklabels=True, yticklabels=True, cbar_kws={'label' : 'correlation'}, ax=ax)

In [26]:
# We see that jan_feb_mar_apr temp are similar, may_june_jul_aug_sep, nov_dec
# train_df['jan_to_apr_avg_temp'] = train_df[['january_avg_temp','february_avg_temp','march_avg_temp']].mean(axis = 1)
# train_df['may_to_sep_avg_temp'] = train_df[['may_avg_temp','june_avg_temp','july_avg_temp','august_avg_temp',\
#                                             'september_avg_temp']].mean(axis = 1)
# train_df['nov_dec_avg_temp'] = train_df[['november_avg_temp','december_avg_temp']].mean(axis = 1)

# test_df['jan_to_apr_avg_temp'] = test_df[['january_avg_temp','february_avg_temp','march_avg_temp']].mean(axis = 1)
# test_df['may_to_sep_avg_temp'] = test_df[['may_avg_temp','june_avg_temp','july_avg_temp','august_avg_temp',\
#                                             'september_avg_temp']].mean(axis = 1)
# test_df['nov_dec_avg_temp'] = test_df[['november_avg_temp','december_avg_temp']].mean(axis = 1)

train_df.shape, test_df.shape

In [27]:
# Label Encoding State_Factor, building_class, and facility_type
le = LabelEncoder()

train_df['State_Factor_le']= le.fit_transform(train_df['State_Factor']).astype("uint8")
test_df['State_Factor_le']= le.fit_transform(test_df['State_Factor']).astype("uint8")

train_df['building_class_le']= le.fit_transform(train_df['building_class']).astype("uint8")
test_df['building_class_le']= le.fit_transform(test_df['building_class']).astype("uint8")

train_df['facility_type_le']= le.fit_transform(train_df['facility_type']).astype("uint8")
test_df['facility_type_le']= le.fit_transform(test_df['facility_type']).astype("uint8")
train_df.shape, test_df.shape

In [31]:
# Looking at average site_eui across state_factor values in train dataset
a = train_df.groupby("State_Factor").agg({'site_eui': 'mean', 'id': 'count'}).reset_index()
a["perc_count"] = (a["id"]/train_df.shape[0])*100
a

In [32]:
# Looking at average site_eui across state_factor values in test dataset
b = test_df.groupby("State_Factor").agg({'id': 'count'}).reset_index()
b["perc_count"] = (b["id"]/test_df.shape[0])*100
b

# Binning State_Factor using the frequency and average site_eui distributions across train and test, with the below bins:
# State_Factors 1 and 10
# State_Factor 11
# State_Factors 6, 4, and 8
# State_Factor 2

In [33]:
# Binning State_Factor based on above logic
train_df["State_Factor_bin"] = train_df["State_Factor"].apply(lambda x: "State_1_10" if x in ["State_1","State_10"]\
                                                              else "State_11" if x in ["State_11"] else "State_6_4_8"
                                                              if x in ["State_6", "State_4", "State_8"] else "State_Others")

test_df["State_Factor_bin"] = test_df["State_Factor"].apply(lambda x: "State_1_10" if x in ["State_1","State_10"]\
                                                              else "State_11" if x in ["State_11"] else "State_6_4_8"
                                                              if x in ["State_6", "State_4", "State_8"] else "State_Others")

In [ ]:
# Looking at the distribution across the State_Factor bins created in train
train_df["State_Factor_bin"].value_counts()/train_df.shape[0]

In [ ]:
# Looking at the distribution across the State_Factor bins created in test
test_df["State_Factor_bin"].value_counts()/test_df.shape[0]

In [34]:
# Label Encoding the binned state_factor variable in train and test

train_df["State_Factor_bin_le"] = le.fit_transform(train_df["State_Factor_bin"]).astype("uint8")
test_df["State_Factor_bin_le"] = le.fit_transform(test_df["State_Factor_bin"]).astype("uint8")

In [35]:
# Dropping State_Factor_bin and State_Factor label encoded

train_df = train_df.drop(columns=['State_Factor_bin','State_Factor_le'], axis=1)
test_df = test_df.drop(columns=['State_Factor_bin','State_Factor_le'], axis=1)
train_df.shape, test_df.shape

In [36]:
# Creating time_since_built for better interpretation of the variable
from datetime import date

train_df["time_since_built"] = date.today().year - train_df["year_built"]
test_df["time_since_built"] = date.today().year - test_df["year_built"]

In [37]:
train_df[["year_built", "time_since_built"]]
train_df.shape, test_df.shape

In [38]:
# Plotting the distribution of time_since_built in train and test
plt.figure(figsize=(8,5))

sns.distplot(train_df["time_since_built"])
sns.distplot(test_df["time_since_built"])        
plt.xlabel("time_since_built")

plt.tight_layout()

In [42]:
train_df.columns

In [43]:
num_features = train_df.select_dtypes(include=['float64']).columns.values

In [44]:
# Correlation
corr = train_df[num_features].corr().abs()

# plot the correlation heatmap

fig, ax = plt.subplots(figsize=(27, 30))
sns.heatmap(corr, vmin=-1, vmax=1, cmap = 'RdBu_r', xticklabels=True, yticklabels=True, cbar_kws={'label' : 'correlation'}, ax=ax)

In [45]:
upper_tri = corr.where(np.triu(np.ones(corr.shape),k=1).astype(np.bool))
upper_tri

In [46]:
# Sort features by their (absolute) correlation with the target variable.

corr.sort_values(by='site_eui', key=abs, ascending=False)['site_eui']

In [47]:
# Looking at Features having >80% correlation with any other feature
[column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]

In [48]:
# Dropping features with >80% correlation 
to_drop = ['february_avg_temp','march_avg_temp','july_avg_temp',"august_avg_temp","september_avg_temp","avg_temp","year_built"]

In [49]:
train_df_new = train_df.drop(columns=train_df[to_drop], axis=1)
test_df_new = test_df.drop(columns=train_df[to_drop], axis=1)

train_df.shape, test_df.shape

In [50]:
train_df_new.shape, test_df_new.shape

In [51]:
train_df_new.select_dtypes(include=["object"])

In [52]:
test_df_new.select_dtypes(include=["object"])

### Feature Scaling

In [55]:
# Splitting train Data into train and validation
train_dta, val_dta = train_test_split(train_df_new, test_size = 0.3)
train_dta.shape, val_dta.shape

In [56]:
# Segmenting data into two segments based on state_factor
train_df_new_sf2 = train_df_new[train_df_new["State_Factor_bin_le"] == 2]
train_df_new_sfrest = train_df_new[train_df_new["State_Factor_bin_le"] != 2]

test_df_new_sf2 = test_df_new[test_df_new["State_Factor_bin_le"] == 2]
test_df_new_sfrest = test_df_new[test_df_new["State_Factor_bin_le"] != 2]
train_df_new_sf2.shape, train_df_new_sfrest.shape, test_df_new_sf2.shape, test_df_new_sfrest.shape

# Dropping columns not required in training
train_df_new_sf2 = train_df_new_sf2.drop(columns=["Year_Factor","id", "facility_type","State_Factor","building_class"], axis=1)
train_df_new_sfrest = train_df_new_sfrest.drop(columns=["Year_Factor","id", "facility_type","State_Factor","building_class"], axis=1)

train_df_new_sf2.shape, train_df_new_sfrest.shape, test_df_new_sf2.shape, test_df_new_sfrest.shape

In [57]:
# train_dta, val_dta = train_df_new[train_df_new["Year_Factor"] < 6], train_df_new[train_df_new["Year_Factor"] == 6]
# Dropping columns not required in training
train_df_new.select_dtypes(include=["object"])

train_dta = train_dta.drop(columns=["Year_Factor","id", "facility_type","State_Factor","building_class"], axis=1)
val_dta = val_dta.drop(columns=["Year_Factor","id", "facility_type","State_Factor","building_class"], axis=1)
test_df_new = test_df_new.drop(columns=["Year_Factor","id", "facility_type","State_Factor","building_class"], axis=1)

In [58]:
test_df_new.shape, train_dta.shape, val_dta.shape

In [59]:
# Creating X and y for train and validation
train_target = train_dta["site_eui"]
val_target = val_dta["site_eui"]

train_dta = train_dta.drop(columns=["site_eui"], axis=1)
val_dta = val_dta.drop(columns=["site_eui"], axis=1)

In [60]:
train_full = train_dta.append(val_dta).reset_index()
target_full = train_target.append(val_target).reset_index()

In [61]:
# Creating X and y for train and validation for segmented data
train_target_sf2 = train_df_new_sf2["site_eui"]
train_target_sfrest = train_df_new_sfrest["site_eui"]

train_dta_sf2 = train_df_new_sf2.drop(columns=["site_eui"], axis=1)
train_dta_sfrest = train_df_new_sfrest.drop(columns=["site_eui"], axis=1)

## Models

In [65]:
from catboost import CatBoostRegressor

### CatBoost

In [67]:
import catboost as cb
from tqdm import tqdm
from sklearn.model_selection import ParameterGrid

RANDOM_STATE = 42

# function returning the average rmse for cross validated test folds
def cross_val(X, y, X_test, param, n_splits=3, n_repeats=3):
    import catboost as cb
    rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=RANDOM_STATE)
    
    rmse = []
    
    for tr_ind, val_ind in rkf.split(X, y):
        X_train = X.iloc[tr_ind]
        X_train = X_train.drop(columns=["index"], axis=1)
        y_train = y.iloc[tr_ind]
        y_train = y_train.drop(columns=["index"], axis=1)
        
        X_valid = X.iloc[val_ind]
        X_valid = X_valid.drop(columns=["index"], axis=1)
        y_valid = y.iloc[val_ind]
        y_valid = y_valid.drop(columns=["index"], axis=1)
        
        catb = cb.CatBoostRegressor(iterations=param['iterations'],
                                depth=param['depth'],
                                l2_leaf_reg = param['l2_leaf_reg'],
                                learning_rate = param['learning_rate'],
                                loss_function = 'RMSE')
                                                 
        catb.fit(X_train, y_train, eval_set=(X_valid, y_valid))
        
        y_pred = catb.predict(X_valid)
        root_mse = np.sqrt(mean_squared_error(y_valid, y_pred))
        rmse.append(root_mse)
    return np.mean(rmse)

In [68]:
# function conducting a gridsearch along with cross validation and returning the best params giving lowest rmse
def catboost_GridSearchCV(X, y, X_test, params, n_splits=3, n_repeats=3):
    ps = {'rmse':100,
          'param': []
    }
    
    
    for prms in tqdm(list(ParameterGrid(params)), ascii=True, desc='Params Tuning:'):
                          
        rmse = cross_val(X, y, X_test, prms, n_splits=n_splits, n_repeats=n_repeats)

        if rmse<ps['rmse']:
            ps['rmse'] = rmse
            ps['param'] = prms
    print('RMSE: '+str(ps['rmse']))
    print('Params: '+str(ps['param']))
    
    return ps['param']

In [69]:
# Declaring the parameters over which grid search will be done
params = {'depth': [4, 7, 10, 12],
          'learning_rate' : [0.03, 0.1, 0.15],
          'l2_leaf_reg': [1,4,9],
          'iterations': [100, 200, 500],
          'early_stopping_rounds': [9],
          'random_seed': [417]}

param = catboost_GridSearchCV(train_full, target_full, test_df_new, params, n_splits=3, n_repeats=3)

cb = cb.CatBoostRegressor()
cb.grid_search(params, train_dta, y=train_target)

In [70]:
param # best parameters

In [71]:
train_full = train_full.drop(columns=["index"], axis=1)
target_full = target_full.drop(columns=["index"], axis=1)

In [73]:
# Fitting the model with the best params from gridsearch with CV
catb_hpo = CatBoostRegressor(
    verbose=1000,
    early_stopping_rounds=9,
    random_seed=417,
    depth=10,
    learning_rate=0.15,
    iterations=500,
    l2_leaf_reg=1,
    loss_function='RMSE'
    )

catb_hpo.fit(train_full, target_full) # fitting the model on full train dataset (since did cross validation, did not split train data into train and validation)
catb_hpo.fit(train_dta_sfrest, train_target_sfrest) # fitting the model on segmented data with state_factor not in 6,4,8

In [74]:
# Grid Search with CV for RF for the other segment (State_factor in 6,4,8)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [10, 90, 100],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 300, 1000]
}
# Create a base model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(train_dta_sf2, train_target_sf2)
rf = RandomForestRegressor(criterion='mse',max_depth = 10, random_state=417)
rf.fit(train_dta_sf2, train_target_sf2)

In [ ]:
grid_search.best_params_ # best params from grid search

In [75]:
# Fitting RF on the other segment with the best params from GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf_best = RandomForestRegressor(bootstrap=True, max_depth=90, max_features=3, min_samples_leaf=3, min_samples_split=12, n_estimators=100)
rf_best.fit(train_dta_sf2, train_target_sf2)

In [76]:
# print("Train Mean Absolute Error: ", mean_absolute_error(train_target, catb.predict(train))) # 25.498
# print("Train Mean Square Error: ", mean_squared_error(train_target, catb.predict(train))) # 2149.902
# print("Train RMSE: ", np.sqrt(mean_squared_error(train_target, catb.predict(train)))) # 46.367

# print("Val Mean Absolute Error: ", mean_absolute_error(val_target, catb_predict)) # 25.498
# print("Val Mean Square Error: ", mean_squared_error(val_target, catb_predict)) # 2149.902
# print("Val RMSE: ", np.sqrt(mean_squared_error(val_target, catb_predict))) # 46.367
# print("RMSE HPO1: ", np.sqrt(mean_squared_error(target_full, catb_hpo.predict(train_full)))) # 28.322
# print("RMSE HPO2: ", np.sqrt(mean_squared_error(target_full, catb_hpo.predict(train_full)))) # 28.23
print("RMSE HPO3: ", np.sqrt(mean_squared_error(target_full, catb_hpo.predict(train_full_scaled)))) # 28.23
print("RMSE HPO SFREST: ", np.sqrt(mean_squared_error(train_target_sfrest, catb_hpo.predict(train_dta_sfrest)))) # 14.64
print("RMSE HPO SF2: ", np.sqrt(mean_squared_error(train_target_sf2, rf_best.predict(train_dta_sf2)))) # 40.6

In [77]:
# Feature importance in the catboost regression
feature_imp = pd.DataFrame(sorted(zip(catb_hpo.feature_importances_, train_full.columns),reverse = True), columns=['Value','Feature'])
feature_imp = feature_imp[feature_imp.Value != 0]
plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('CatBoost Feature Importance')
plt.tight_layout()
plt.show()
plt.savefig('catb_importances-01.png')

### Kaggle Test Prediction & Submission

In [78]:
# test_df_x = test_df[x_train.columns]
test_df_final = pd.read_csv("../input/widsdatathon2022/test.csv")
test_df_final.head()

In [79]:
# Dropping unnecessary columns
test_df_new_sf2_v2 = test_df_new_sf2.drop(columns=["Year_Factor","id", "facility_type","State_Factor","building_class"], axis=1)
test_df_new_sfrest_v2 = test_df_new_sfrest.drop(columns=["Year_Factor","id", "facility_type","State_Factor","building_class"], axis=1)

# Predicting site_eui for both the segments
predict_sub_sfrest = catb_hpo.predict(test_df_new_sfrest_v2)
predict_sub_sf2 = rf_best.predict(test_df_new_sf2_v2)

In [80]:
# train_dta_sf2.columns, test_df_new_sf2_v2.columns

In [81]:
test_df_new_sfrest["pred"] = predict_sub_sfrest
test_df_new_sf2["pred"] = predict_sub_sf2
# test_df_new_sfrest.head(), test_df_new_sf2.head()

In [82]:
# Merging the predictions with the full test data using id, year_factor, state_factor as keys
test_df_final_pred1=test_df_final.merge(test_df_new_sfrest, on=["id","Year_Factor","State_Factor"], how='left')
test_df_final_pred2=test_df_final_pred1.merge(test_df_new_sf2, on=["id","Year_Factor","State_Factor"], how='left')

In [83]:
# Creating final pred column taking the prediction of the respective model
test_df_final_pred2['pred'] = np.where(test_df_final_pred2.pred_x.isna(),test_df_final_pred2.pred_y,test_df_final_pred2.pred_x)

In [84]:
# Final prediction
predict_sub = test_df_final_pred2['pred']

In [85]:
sample_sol = pd.read_csv("../input/widsdatathon2022/sample_solution.csv")

In [86]:
len(predict_sub), len(sample_sol.site_eui)

In [87]:
sample_sol.site_eui = predict_sub

In [88]:
sample_sol.to_csv("submission_catb_vritti_eco_wochigo_hpo.csv", index=False)